In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import sympy as sp

## Required function

In [ ]:
def CompositeSimposonInt(x,f):
    dx = (x[1]-x[0])
    approx = 1/3*dx*(f[0]+4*np.sum(f[1::2])+2*sum(f[2::2][:-1])+f[-1])
    return approx

def eta0(w):
    return np.sum(w) * dx

def eta1(w):
    return 0.5 * np.sum(w*w) * dx

def eta2(w):
    f1 = np.append(w,w[0])**3
    what = np.fft.fft(w)
    wx = np.real(np.fft.ifft(1j*xi*what))
    f2 = np.append(wx,wx[0])**2
    int_f1 = CompositeSimposonInt(xplot,f1)
    int_f2 = CompositeSimposonInt(xplot,f2)
    return 2*int_f1 - int_f2

## Loading data

In [ ]:
# Change here to get numerical results for different cases
sol = 3; inv = 2;
Data_Folder = 'Data'; Fig_Folder = 'Figures'

if sol == 2 and inv == 2: 
    eqn = 'KdV_%d_sol_%d_inv'%(2,2);
    xL = -80; xR = 80; L = xR-xL; N = 1024; DT = [0.01, 0.01]; t0 = -25; T = 25
    
elif sol == 3 and inv == 2:
    eqn = 'KdV_%d_sol_%d_inv'%(3,2);
    xL = -130; xR = 130; L = xR-xL; N = 1536; DT = [0.01, 0.01]; t0 = -50; T = 50

# method data
df = pd.read_csv("%s/%s/MethodsData_N%d_T%d.csv"%(Data_Folder,eqn,N,T),index_col=[0])
# Numerical Solution
b_tt = np.load("%s/%s/Baseline_Time_N%d_T%d.npy"%(Data_Folder,eqn,N,T), allow_pickle=True)
b_uu = np.load("%s/%s/Baseline_NumSol_N%d_T%d.npy"%(Data_Folder,eqn,N,T), allow_pickle=True)
r_tt = np.load("%s/%s/Relaxation_Time_N%d_T%d.npy"%(Data_Folder,eqn,N,T), allow_pickle=True)
r_uu = np.load("%s/%s/Relaxation_NumSol_N%d_T%d.npy"%(Data_Folder,eqn,N,T), allow_pickle=True)
# Reference Solution
b_UTrue = np.load("%s/%s/TrueSol_BaselineTime_N%d_T%d.npy"%(Data_Folder,eqn,N,T), allow_pickle=True)
r_UTrue = np.load("%s/%s/TrueSol_RelaxationTime_N%d_T%d.npy"%(Data_Folder,eqn,N,T), allow_pickle=True)

#-------------------------------------------------------------------------------#
# python program to check if a path exists, if path doesn’t exist we create a new path
import os
path = '%s/%s'%(Fig_Folder,eqn)

import os
if not os.path.exists(path):
   os.makedirs(path)
#-------------------------------------------------------------------------------#

In [ ]:
# Info table
df

In [ ]:
df['Method'] = ['ARK3(2)4L[2]SA','ARK4(3)4L[2]SA']
df['Mthd_Save_Name'] = ['ARK32','ARK43']

In [ ]:
# Domain
xL = int(eval(df['Domain'][0])[0]) # left end point of the domain 
xR = int(eval(df['Domain'][0])[1]) # right end point of the domain 
L = xR-xL # length of the domain
N = df['N'][0] # number of grid points
xplot = np.linspace(xL, xR, N+1)
x = xplot[0:-1] 
dx = x[1] - x[0]
xi = np.fft.fftfreq(N) * N * 2*np.pi / L

## Plotting solution

## Invariants

In [ ]:
b_ETA_0 = []; r_ETA_0 = []; b_ETA_1 = []; r_ETA_1 = []; b_ETA_2 = []; r_ETA_2 = [];
for i in range(len(df['Method'])):
    b_eta_0 = [eta0(u) for u in b_uu[i]] - eta0(b_uu[i][0])
    b_eta_1 = [eta1(u) for u in b_uu[i]] - eta1(b_uu[i][0])
    b_eta_2 = [eta2(u) for u in b_uu[i]] - eta2(b_uu[i][0])
    r_eta_0 = [eta0(u) for u in r_uu[i]] - eta0(r_uu[i][0])
    r_eta_1 = [eta1(u) for u in r_uu[i]] - eta1(r_uu[i][0])
    r_eta_2 = [eta2(u) for u in r_uu[i]] - eta2(r_uu[i][0])
    b_ETA_0.append(b_eta_0);
    r_ETA_0.append(r_eta_0)
    b_ETA_1.append(b_eta_1); 
    r_ETA_1.append(r_eta_1)
    b_ETA_2.append(b_eta_2); 
    r_ETA_2.append(r_eta_2)

In [ ]:
# plotting invariants
# Font size    
font = {#'family' : 'normal',
'weight' : 'normal',
'size'   : 14}
plt.rc('font', **font)
plt.figure(figsize=(15, 4))

for i in range(len(df['Method'])):
    plt.subplot(1,len(df['Method']),i+1)
    plt.plot(b_tt[i],b_ETA_0[i],':k',label="Baseline: $\eta_{0}(U(t))-\eta_{0}(U(0))$")
    plt.plot(r_tt[i],r_ETA_0[i],'-r',label="Relaxation: $\eta_{0}(U(t))-\eta_{0}(U(0))$")
    plt.plot(b_tt[i],b_ETA_1[i],':b',label="Baseline: $\eta_{1}(U(t))-\eta_{1}(U(0))$")
    plt.plot(r_tt[i],r_ETA_1[i],'-g',label="Relaxation: $\eta_{1}(U(t))-\eta_{1}(U(0))$")
    plt.plot(b_tt[i],b_ETA_2[i],':m',label="Baseline: $\eta_{2}(U(t))-\eta_{2}(U(0))$")
    plt.plot(r_tt[i],r_ETA_2[i],'-y',label="Relaxation: $\eta_{2}(U(t))-\eta_{2}(U(0))$")
    
    plt.title('%s with $\Delta t$ = %.2f'%(df['Method'][i],df['B: dt'][i]))
    plt.xlabel('$t$')
    plt.yscale("symlog", linthresh=1.e-14)
    plt.yticks([-1.e-2, -1.e-6, -1.e-10, -1.e-14, 1.e-14, 1.e-10, 1.e-6, 1.e-2])
    
plt.tight_layout()
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.2))
plt.savefig('%s/%s/%s_InvariantVsTime_dt%1.0e_tf%d.pdf'%(Fig_Folder,eqn,eqn,df['B: dt'][i],T),format='pdf', bbox_inches="tight",transparent=True)


## Max of invariants error

In [ ]:
for i in range(len(df['Method'])):
    print("Baseline %s: Max of mass invariant error = %1.2e. \n"%(df['Method'][i],np.max(np.abs(b_ETA_0[i]))))
    print("Relaxation %s: Max of mass invariant error = %1.2e. \n"%(df['Method'][i],np.max(np.abs(r_ETA_0[i]))))

    print("Baseline %s: Max of energy invariant error = %1.2e. \n"%(df['Method'][i],np.max(np.abs(b_ETA_1[i]))))
    print("Relaxation %s: Max of energy invariant error = %1.2e. \n"%(df['Method'][i],np.max(np.abs(r_ETA_1[i]))))

    print("Baseline %s: Max of Whitham invariant error = %1.2e. \n"%(df['Method'][i],np.max(np.abs(b_ETA_2[i]))))
    print("Relaxation %s: Max of Whitham invariant error = %1.2e. \n"%(df['Method'][i],np.max(np.abs(r_ETA_2[i]))))

## Errors

In [ ]:
b_ERR = []; r_ERR = []
for i in range(len(df['Method'])):
    b_uexact = b_UTrue[i]
    b_err = np.max(np.abs(b_uu[i]-b_uexact),axis=1)
    r_uexact = r_UTrue[i]
    r_err = np.max(np.abs(r_uu[i]-r_uexact),axis=1)
    b_ERR.append(b_err); 
    r_ERR.append(r_err) 

In [ ]:
# Plotting
if sol == 2 and inv == 2:
    sl2_cons_mult = [4e-6,2e-8]; sl2_p = [2,2]
    sl1_cons_mult = [10e-5,5e-8]; sl1_p = [1,1] 
    slope_st_pt = 5  
    
elif sol == 3 and inv == 2:
    sl2_cons_mult = [5e-6,2e-8,2e-9]; sl2_p = [2,2,2]
    sl1_cons_mult = [9e-5,5e-8,20e-9]; sl1_p = [1,1,1]  
    slope_st_pt = 10

    
# Font size    
font = {#'family' : 'normal',
'weight' : 'normal',
'size'   : 14}
plt.rc('font', **font)
plt.figure(figsize=(15, 4))

for i in range(len(df['Method'])):
    plt.subplot(1,len(df['Method']),i+1)
    plt.plot(-df['t0'][0]+b_tt[i],b_ERR[i],':',color='orangered',label="Baseline")
    plt.plot(-df['t0'][0]+r_tt[i],r_ERR[i],'-g',label="Relaxation")
    sl_b = np.linspace(slope_st_pt,-df['t0'][0]+df['tf'][0],100)
    sl_r = np.linspace(slope_st_pt,-df['t0'][0]+df['tf'][0],100)

    plt.plot(sl_r,sl1_cons_mult[i]*sl_r**sl1_p[i],'--',color='0.5',label="$\mathcal{O}(t^{%1.1f})$"%(sl1_p[i]))
    plt.plot(sl_b,sl2_cons_mult[i]*sl_b**sl2_p[i],'-',color='0.5',label="$\mathcal{O}(t^{%1.1f})$"%(sl2_p[i]))
    
    plt.xlabel('$t+%d$'%(-df['t0'][0]))
    plt.ylabel('Error in u')
    plt.xscale("log"); plt.yscale("log")
    #plt.yticks([ -1.e-1, -1.e-2,-1.e-4, 1.e-4, 1.e-2,1.e-1])
    plt.title('%s with $\Delta t$ = %.2f'%(df['Method'][i],df['B: dt'][i]))
    
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc='upper center', ncol=4, bbox_to_anchor=(0.5, 1.15))
plt.savefig('%s/%s/%s_ErrorVsTime_dt%1.0e_tf%d.pdf'%(Fig_Folder,eqn,eqn,df['B: dt'][i],T),format='pdf', bbox_inches="tight")